# Flights optimization

## 4. Exploring the patterns

### TASK 4.1. 
База данных содержит список аэропортов практически всех крупных городов России. В большинстве городов есть только один аэропорт. Исключение составляет: 

**Answer:** Moscow, Ulyanovsk

SELECT city,
       count(city) AS qty
FROM dst_project.airports
GROUP BY 1
ORDER BY 2 DESC

### TASK 4.2. 

**Question 1** 

Таблица рейсов содержит всю информацию о прошлых, текущих и запланированных рейсах. Сколько всего статусов для рейсов определено в таблице?

**Answer:** 6

SELECT count(DISTINCT status)
FROM dst_project.flights

**Question 2** 

Какое количество самолетов находятся в воздухе на момент среза в базе (статус рейса «самолёт уже вылетел и находится в воздухе»)?

**Answer:** 58

SELECT count(*)
FROM dst_project.flights
WHERE status = 'Departed'

**Question 3** 

Места определяют схему салона каждой модели. Сколько мест имеет самолет модели  (Boeing 777-300)?

**Answer:** 402

SELECT count(*)
FROM dst_project.seats
JOIN dst_project.aircrafts 
    USING(aircraft_code)
WHERE model = 'Boeing 777-300'

**Question 4** 

Сколько состоявшихся (фактических) рейсов было совершено между 1 апреля 2017 года и 1 сентября 2017 года?

**Answer:** 74227

SELECT count(*)
FROM dst_project.flights
WHERE status = 'Arrived'
  AND actual_arrival BETWEEN '2017-04-01' AND '2017-09-01'

### TASK 4.3. 

**Question 1** 

Сколько всего рейсов было отменено по данным базы?

**Answer:** 437

SELECT count(*) /* выводим количество */
FROM dst_project.flights 
WHERE status = 'Cancelled' /* фильтрация результатов по отмене рейса */

**Question 2** 

Сколько самолетов моделей типа Boeing, Sukhoi Superjet, Airbus находится в базе авиаперевозок?

**Answer:** 3, 1, 3

SELECT 
    (SELECT COUNT(*) /* выводим количество самолётов, отобранных в запросе ниже */
    FROM dst_project.aircrafts /* берём данные из таблицы "Самолёты" */
    WHERE model like ('Boeing%')) AS Boeing, /* отбираем данные по всем модификациям Boeing */
    (SELECT COUNT(*)
    FROM dst_project.aircrafts
    WHERE model like ('Sukhoi%')) AS Sukhoi_Superjet, /* отбираем данные по всем модификациям Sukhoi Superjet */
    (SELECT COUNT(*)
    FROM dst_project.aircrafts
    WHERE model like ('Airbus%')) AS Airbus /* отбираем данные по всем модификациям Airbus */
    
FROM dst_project.aircrafts
LIMIT 1;

P.S. В условии задания речь идёт о базе авиаперевозок, а не базе самолётов. Если брать значения по базе авиаперевозок, то в ней представлены только 2 модели Airbus вместо 3 в базе самолётов.

SELECT count(DISTINCT aircraft_code)
FROM dst_project.flights
WHERE aircraft_code in
    (SELECT aircraft_code
     FROM dst_project.aircrafts
     WHERE model like ('Airbus%'))

 **Question 3** 

В какой части (частях) света находится больше аэропортов?

**Answer:** Europe, Asia

SELECT
  (SELECT count(*) /* выводим количество */
   FROM dst_project.airports /* берём данные из таблицы "Аэропорты" */
   WHERE timezone like ('Asia%') ) AS Asia, /* прописываем условие принадлежности аэропорта к Азии */

  (SELECT count(*) 
   FROM dst_project.airports
   WHERE timezone like ('Europe%') ) AS Europe, /* прописываем условие принадлежности аэропорта к Европе */

  (SELECT count(*)
   FROM dst_project.airports
   WHERE timezone like ('Australia%') ) AS Australia, /* прописываем условие принадлежности аэропорта к Австралии */
       Count(*) AS Total /* считаем общее количество аэропортов, напрямую по списку в исходной таблице */
FROM dst_project.airports
LIMIT 1;
    

 **Question 4** 

У какого рейса была самая большая задержка прибытия за все время сбора данных? Введите id рейса (flight_id)

**Answer:** 157571

SELECT flight_id,
       actual_arrival - scheduled_arrival AS max_delay /* выводим конкретную задержку */
FROM dst_project.flights
WHERE actual_arrival IS NOT NULL /* берём данные только в тех кортежах, где указано время прибытия */
ORDER BY 2 DESC /* фильтруем данные по убыванию, чтобы максимальная задержка расположилась первой */
LIMIT 1;

### TASK 4.4. 

**Question 1** 

Когда был запланирован самый первый вылет, сохраненный в базе данных?

**Answer:** 14.08.2016

SELECT scheduled_departure /* выводим данные запланированного вылета */
FROM dst_project.flights 
ORDER BY 1 /* фильтруем данные по возрастанию, чтобы данные первый запланированный вылет расположился первым */
LIMIT 1;

**Question 2** 

Сколько минут составляет запланированное время полета в самом длительном рейсе?

**Answer:** 530

SELECT date_part('hour', scheduled_arrival - scheduled_departure) * 60 
    + date_part('minute', scheduled_arrival - scheduled_departure) 
    /* выводим общее количество минут запланированного полёта */
FROM dst_project.flights
ORDER BY 1 DESC /* фильтруем значения по убыванию, чтобы в начале отобразился самый продолжительный запланированный полёт */
LIMIT 1 /* выводим только одно первое значение, больше не нужно */;

**Question 3** 

Между какими аэропортами пролегает самый длительный по времени запланированный рейс?

**Answer:** DME - UUS

SELECT departure_airport,
       arrival_airport /* выводим коды аэропортов вылета и прилета */
FROM dst_project.flights
WHERE scheduled_arrival - scheduled_departure in 
/* прописываем условие по которому будем отбирать максимальную запланированную продолжительность полёта */
    (SELECT scheduled_arrival - scheduled_departure
     FROM dst_project.flights
     ORDER BY 1 DESC /* фильтруем значения по убыванию, чтобы в начале отобразился самый продолжительный запланированный полёт */
     LIMIT 1 /* выводим только одно первое значение, больше не нужно */)
LIMIT 1;

**Question 4** 

 Сколько составляет средняя дальность полета среди всех самолетов в минутах? Секунды округляются в меньшую сторону (отбрасываются до минут).

**Answer:** 128

SELECT date_part('hour', avg(f.scheduled_arrival - f.scheduled_departure)) * 60 
    + date_part('minute', avg(f.scheduled_arrival - f.scheduled_departure))
/* выводим сумму в минутах средней по времени продолжительности запланированного полета */
/* секундны не берём в расчёт, так как по условию задания они отбрасываются (поэтому и не учитываем 21 секунду) */
FROM dst_project.flights f; 

### TASK 4.5. 

**Question 1** 

Мест какого класса у SU9 больше всего?

**Answer:** Economy

SELECT fare_conditions,
       count(fare_conditions) /* выводим классы и количество по мест по ним */
FROM dst_project.seats
WHERE aircraft_code = 'SU9' /* используем условие соответствия к конкретному id модели самолёта */
GROUP BY 1;

**Question 2** 

Какую самую минимальную стоимость составило бронирование за всю историю?

**Answer:** 3400

SELECT total_amount
FROM dst_project.bookings
ORDER BY 1 ASC
LIMIT 1;

**Question 3** 

Какой номер места был у пассажира с id = 4313 788533?

**Answer:** 2A

SELECT bp.seat_no /* выводим номер места */
FROM dst_project.tickets t
JOIN dst_project.boarding_passes bp USING(ticket_no) /* объединяем таблицы tickets и boarding_passes */
WHERE t.passenger_id = '4313 788533' /* прописываем условие по конкретному id пассажира*/;

## 5. Preliminary analysis

### TASK 5.1. 

**Question 1** 

Анапа — курортный город на юге России. Сколько рейсов прибыло в Анапу за 2017 год?

**Answer:** 486

SELECT count(*) /* выводим количество */
FROM dst_project.flights
WHERE arrival_airport in
/* прописываем условие соответствия записей коду аэропорта Анапы и году прибытия - 2017 */
    (SELECT airport_code
    FROM dst_project.airports
    WHERE city = 'Anapa')
    AND date_part('year', actual_arrival) = 2017;

**Question 2** 

Сколько рейсов из Анапы вылетело зимой 2017 года?

**Answer:** 127

SELECT count(*)
FROM dst_project.flights
WHERE departure_airport in
/* прописываем условие соответствия записей коду аэропорта Анапы и году вылета 2017 + месяцам вылета - декабрю, январю и февралю + исключаем отменённые рейсы */
    (SELECT airport_code
     FROM dst_project.airports
     WHERE city = 'Anapa')
  AND date_part('year', actual_departure) = 2017
  AND date_part('month', actual_departure) in (12,1,2)
  AND status not in ('Cancelled');

**Question 3** 

Посчитайте количество отмененных рейсов из Анапы за все время.

**Answer:** 

SELECT count(*)
FROM dst_project.flights
WHERE departure_airport in
/* прописываем условие соответствия записей коду аэропорта Анапы и статусы вылета рейса "Отменён" */
    (SELECT airport_code
     FROM dst_project.airports
     WHERE city = 'Anapa')
  AND status = 'Cancelled';

**Question 4** 

Сколько рейсов из Анапы не летают в Москву?

**Answer:** 453

SELECT count(*)
FROM dst_project.flights
WHERE departure_airport in
/* прописываем условия соответствия записей коду аэропорта Анапы (вылет) и отсутствию совпадений с кодами аэропортов Москвы (прилёт) */
    (SELECT airport_code
    FROM dst_project.airports
    WHERE city = 'Anapa') 
    AND arrival_airport not in 
    (SELECT airport_code
    FROM dst_project.airports
    WHERE city = 'Moscow')

**Question 5** 

Какая модель самолета летящего на рейсах из Анапы имеет больше всего мест?

**Answer:** Boeing 737-300

SELECT *
FROM
/* соединяем две таблицы с вложенными запросами: */
  (SELECT DISTINCT model,
                   aircraft_code /* первая таблица - выводим уникальные названия моделей и коды самолётов */
   FROM dst_project.flights
   INNER JOIN dst_project.aircrafts USING(aircraft_code) /* соединяем две таблицы - полёты и самолёты */
   WHERE departure_airport in /* прописываем условие соответствия аэропорта вылета Анапе */
       (SELECT airport_code
        FROM dst_project.airports
        WHERE city = 'Anapa')) AS models
INNER JOIN
  (SELECT DISTINCT aircraft_code,
                   count(*) /* вторая таблица - выводим уникальные коды самолётов и подсчитываем количество мест в каждой модели */
   FROM dst_project.seats
   GROUP BY 1) AS seats_qty USING(aircraft_code)
ORDER BY 3 DESC /* фильтруем данные по количеству мест в самолётах по убыванию */;